---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [2]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    import pandas as pd
    import numpy as np
    with open('university_towns.txt') as f:
        university_towns = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    university_towns = [x.strip() for x in university_towns] 
    lst = []
    state = ''
    region = ''
    for line in university_towns:
        if '[ed' in line:
            state = line.split('[')[0]
        else:
            region = line.split(' (')[0].strip()
            lst.append([state, region]) 
    df = pd.DataFrame(lst, columns=['State','RegionName'])

    return df



In [7]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''

    import pandas as pd
    import numpy as np
    from scipy.stats import ttest_ind

    gdp = pd.read_excel('gdplev.xls', parse_cols="E:G", skiprows=218, skip_footer=0, na_values='...')
    gdp.columns = ['Quarter', 'Current Dollars', 'Chained Dollars']
    gdp['GDP change'] = gdp['Chained Dollars'] - gdp['Chained Dollars'].shift(+1)  
    gdp['GDP change2'] = gdp['Chained Dollars'] - gdp['Chained Dollars'].shift(-1)
    gdp['Recession'] = (gdp['GDP change'] < 0) & (gdp['GDP change2'] > 0)
    recession = gdp.loc[gdp['Recession'] == 1]
    recession = recession.set_index('Quarter')
    return recession.index[0]


In [14]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    from scipy.stats import ttest_ind

    gdp = pd.read_excel('gdplev.xls', parse_cols="E:G", skiprows=218, skip_footer=0, na_values='...')
    gdp.columns = ['Quarter', 'Current Dollars', 'Chained Dollars']
    gdp['GDP change -1'] = gdp['Chained Dollars'].shift(+1) - gdp['Chained Dollars'].shift(+2) 
    gdp['GDP change'] = gdp['Chained Dollars'] - gdp['Chained Dollars'].shift(+1)  
    gdp['GDP change +1'] = gdp['Chained Dollars'].shift(-1) - gdp['Chained Dollars']
    gdp['GDP surge'] = (gdp['GDP change'] > 0) & (gdp['GDP change -1'] > 0)
    gdp['Recession Start'] = (gdp['GDP change'] < 0) & (gdp['GDP change +1'] < 0)
    gdp['Recession On'] = (gdp['Recession Start'] == 1) | (gdp['Recession Start'].shift(+1) == 1) | ((gdp['Recession Start'].shift(+2)==1) & (gdp['GDP surge']==0))
    gdp['Recession End'] = (gdp['Recession On']==0) & (gdp['Recession On'].shift(+1)==1)
    recession_end = gdp.loc[gdp['Recession End'] == 1]
    recession_end = recession_end.set_index('Quarter')
 
    return recession_end.index[0]


,Current Dollars,Chained Dollars,GDP change -1,GDP change,GDP change +1,GDP surge,Recession Start,Recession On,Recession End
Quarter,,,,,,,,,
2009q4,14566.5,14541.9,46.9,139.4,62.9,True,False,False,True


In [13]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    from scipy.stats import ttest_ind

    gdp = pd.read_excel('gdplev.xls', parse_cols="E:G", skiprows=218, skip_footer=0, na_values='...')
    gdp.columns = ['Quarter', 'Current Dollars', 'Chained Dollars']
    gdp['GDP change -1'] = gdp['Chained Dollars'].shift(+1) - gdp['Chained Dollars'].shift(+2) 
    gdp['GDP change'] = gdp['Chained Dollars'] - gdp['Chained Dollars'].shift(+1)  
    gdp['GDP change +1'] = gdp['Chained Dollars'].shift(-1) - gdp['Chained Dollars']
    gdp['GDP surge'] = (gdp['GDP change'] > 0) & (gdp['GDP change -1'] > 0)
    gdp['Recession Start'] = (gdp['GDP change'] < 0) & (gdp['GDP change +1'] < 0)
    gdp['Recession On'] = (gdp['Recession Start'] == 1) | (gdp['Recession Start'].shift(+1) == 1) | ((gdp['Recession Start'].shift(+2)==1) & (gdp['GDP surge']==0))
    recession1 = gdp.loc[gdp['Recession On']==1]
    recession1 = recession1.sort_values('Chained Dollars')
    recession1 = recession1.set_index('Quarter')

    return recession1.index[0]

In [51]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import pandas as pd
    import numpy as np
    from scipy.stats import ttest_ind

    range1 = [i for i in range(1,5)]
    range2 = [i for i in range(51,251)]
    usecols = range1 + range2
    housing = pd.read_csv('City_Zhvi_AllHomes.csv', skiprows=0, usecols=usecols, na_values='...')
    housing['2000q1'] = housing.iloc[:,4:7].mean(axis=1)
    housing['2000q2'] = housing.iloc[:,7:10].mean(axis=1)
    housing['2000q3'] = housing.iloc[:,10:13].mean(axis=1)
    housing['2000q4'] = housing.iloc[:,13:16].mean(axis=1)
    housing['2001q1'] = housing.iloc[:,16:19].mean(axis=1)
    housing['2001q2'] = housing.iloc[:,19:22].mean(axis=1)
    housing['2001q3'] = housing.iloc[:,22:25].mean(axis=1)
    housing['2001q4'] = housing.iloc[:,25:28].mean(axis=1)
    housing['2002q1'] = housing.iloc[:,28:31].mean(axis=1)
    housing['2002q2'] = housing.iloc[:,31:34].mean(axis=1)
    housing['2002q3'] = housing.iloc[:,34:37].mean(axis=1)
    housing['2002q4'] = housing.iloc[:,37:40].mean(axis=1)
    housing['2003q1'] = housing.iloc[:,40:43].mean(axis=1)
    housing['2003q2'] = housing.iloc[:,43:46].mean(axis=1)
    housing['2003q3'] = housing.iloc[:,46:49].mean(axis=1)
    housing['2003q4'] = housing.iloc[:,49:52].mean(axis=1)
    housing['2004q1'] = housing.iloc[:,52:55].mean(axis=1)
    housing['2004q2'] = housing.iloc[:,55:58].mean(axis=1)
    housing['2004q3'] = housing.iloc[:,58:61].mean(axis=1)
    housing['2004q4'] = housing.iloc[:,61:64].mean(axis=1)
    housing['2005q1'] = housing.iloc[:,64:67].mean(axis=1)
    housing['2005q2'] = housing.iloc[:,67:70].mean(axis=1)
    housing['2005q3'] = housing.iloc[:,70:73].mean(axis=1)
    housing['2005q4'] = housing.iloc[:,73:76].mean(axis=1)
    housing['2006q1'] = housing.iloc[:,76:79].mean(axis=1)
    housing['2006q2'] = housing.iloc[:,79:82].mean(axis=1)
    housing['2006q3'] = housing.iloc[:,82:85].mean(axis=1)
    housing['2006q4'] = housing.iloc[:,85:88].mean(axis=1)
    housing['2007q1'] = housing.iloc[:,88:91].mean(axis=1)
    housing['2007q2'] = housing.iloc[:,91:94].mean(axis=1)
    housing['2007q3'] = housing.iloc[:,94:97].mean(axis=1)
    housing['2007q4'] = housing.iloc[:,97:100].mean(axis=1)
    housing['2008q1'] = housing.iloc[:,100:103].mean(axis=1)
    housing['2008q2'] = housing.iloc[:,103:106].mean(axis=1)
    housing['2008q3'] = housing.iloc[:,106:109].mean(axis=1)
    housing['2008q4'] = housing.iloc[:,109:112].mean(axis=1)
    housing['2009q1'] = housing.iloc[:,112:115].mean(axis=1)
    housing['2009q2'] = housing.iloc[:,115:118].mean(axis=1)
    housing['2009q3'] = housing.iloc[:,118:121].mean(axis=1)
    housing['2009q4'] = housing.iloc[:,121:124].mean(axis=1)
    housing['2010q1'] = housing.iloc[:,124:127].mean(axis=1)
    housing['2010q2'] = housing.iloc[:,127:130].mean(axis=1)
    housing['2010q3'] = housing.iloc[:,130:133].mean(axis=1)
    housing['2010q4'] = housing.iloc[:,133:136].mean(axis=1)
    housing['2011q1'] = housing.iloc[:,136:139].mean(axis=1)
    housing['2011q2'] = housing.iloc[:,139:142].mean(axis=1)
    housing['2011q3'] = housing.iloc[:,142:145].mean(axis=1)
    housing['2011q4'] = housing.iloc[:,145:148].mean(axis=1)
    housing['2012q1'] = housing.iloc[:,148:151].mean(axis=1)
    housing['2012q2'] = housing.iloc[:,151:154].mean(axis=1)
    housing['2012q3'] = housing.iloc[:,154:157].mean(axis=1)
    housing['2012q4'] = housing.iloc[:,157:160].mean(axis=1)
    housing['2013q1'] = housing.iloc[:,160:163].mean(axis=1)
    housing['2013q2'] = housing.iloc[:,163:166].mean(axis=1)
    housing['2013q3'] = housing.iloc[:,166:169].mean(axis=1)
    housing['2013q4'] = housing.iloc[:,169:172].mean(axis=1)
    housing['2014q1'] = housing.iloc[:,172:175].mean(axis=1)
    housing['2014q2'] = housing.iloc[:,175:178].mean(axis=1)
    housing['2014q3'] = housing.iloc[:,178:181].mean(axis=1)
    housing['2014q4'] = housing.iloc[:,181:184].mean(axis=1)
    housing['2015q1'] = housing.iloc[:,184:187].mean(axis=1)
    housing['2015q2'] = housing.iloc[:,187:190].mean(axis=1)
    housing['2015q3'] = housing.iloc[:,190:193].mean(axis=1)
    housing['2015q4'] = housing.iloc[:,193:196].mean(axis=1)
    housing['2016q1'] = housing.iloc[:,196:199].mean(axis=1)
    housing['2016q2'] = housing.iloc[:,199:202].mean(axis=1)
    housing['2016q3'] = housing.iloc[:,202:204].mean(axis=1)
    housing2 = housing.iloc[:,0:2]
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    housing2['State'].replace(states, inplace=True)
    housing3 = housing.iloc[:,204:]
    housing4 = housing2.join(housing3, how='outer')
    housing4 = housing4.set_index(['State','RegionName'])

    return housing4


2010q3        2015q2  2016q4
State RegionName                                    
Ohio  Akron       67566.666667  61833.333333     NaN
      Dayton      57600.000000  50000.000000     NaN

In [1]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    import pandas as pd
    import numpy as np
    from scipy.stats import ttest_ind

    range1 = [i for i in range(1,5)]
    range2 = [i for i in range(51,251)]
    usecols = range1 + range2
    housing = pd.read_csv('City_Zhvi_AllHomes.csv', skiprows=0, usecols=usecols, na_values='...')
    housing['2000q1'] = housing.iloc[:,4:7].mean(axis=1)
    housing['2000q2'] = housing.iloc[:,7:10].mean(axis=1)
    housing['2000q3'] = housing.iloc[:,10:13].mean(axis=1)
    housing['2000q4'] = housing.iloc[:,13:16].mean(axis=1)
    housing['2001q1'] = housing.iloc[:,16:19].mean(axis=1)
    housing['2001q2'] = housing.iloc[:,19:22].mean(axis=1)
    housing['2001q3'] = housing.iloc[:,22:25].mean(axis=1)
    housing['2001q4'] = housing.iloc[:,25:28].mean(axis=1)
    housing['2002q1'] = housing.iloc[:,28:31].mean(axis=1)
    housing['2002q2'] = housing.iloc[:,31:34].mean(axis=1)
    housing['2002q3'] = housing.iloc[:,34:37].mean(axis=1)
    housing['2002q4'] = housing.iloc[:,37:40].mean(axis=1)
    housing['2003q1'] = housing.iloc[:,40:43].mean(axis=1)
    housing['2003q2'] = housing.iloc[:,43:46].mean(axis=1)
    housing['2003q3'] = housing.iloc[:,46:49].mean(axis=1)
    housing['2003q4'] = housing.iloc[:,49:52].mean(axis=1)
    housing['2004q1'] = housing.iloc[:,52:55].mean(axis=1)
    housing['2004q2'] = housing.iloc[:,55:58].mean(axis=1)
    housing['2004q3'] = housing.iloc[:,58:61].mean(axis=1)
    housing['2004q4'] = housing.iloc[:,61:64].mean(axis=1)
    housing['2005q1'] = housing.iloc[:,64:67].mean(axis=1)
    housing['2005q2'] = housing.iloc[:,67:70].mean(axis=1)
    housing['2005q3'] = housing.iloc[:,70:73].mean(axis=1)
    housing['2005q4'] = housing.iloc[:,73:76].mean(axis=1)
    housing['2006q1'] = housing.iloc[:,76:79].mean(axis=1)
    housing['2006q2'] = housing.iloc[:,79:82].mean(axis=1)
    housing['2006q3'] = housing.iloc[:,82:85].mean(axis=1)
    housing['2006q4'] = housing.iloc[:,85:88].mean(axis=1)
    housing['2007q1'] = housing.iloc[:,88:91].mean(axis=1)
    housing['2007q2'] = housing.iloc[:,91:94].mean(axis=1)
    housing['2007q3'] = housing.iloc[:,94:97].mean(axis=1)
    housing['2007q4'] = housing.iloc[:,97:100].mean(axis=1)
    housing['2008q1'] = housing.iloc[:,100:103].mean(axis=1)
    housing['2008q2'] = housing.iloc[:,103:106].mean(axis=1)
    housing['2008q3'] = housing.iloc[:,106:109].mean(axis=1)
    housing['2008q4'] = housing.iloc[:,109:112].mean(axis=1)
    housing['2009q1'] = housing.iloc[:,112:115].mean(axis=1)
    housing['2009q2'] = housing.iloc[:,115:118].mean(axis=1)
    housing['2009q3'] = housing.iloc[:,118:121].mean(axis=1)
    housing['2009q4'] = housing.iloc[:,121:124].mean(axis=1)
    housing['2010q1'] = housing.iloc[:,124:127].mean(axis=1)
    housing['2010q2'] = housing.iloc[:,127:130].mean(axis=1)
    housing['2010q3'] = housing.iloc[:,130:133].mean(axis=1)
    housing['2010q4'] = housing.iloc[:,133:136].mean(axis=1)
    housing['2011q1'] = housing.iloc[:,136:139].mean(axis=1)
    housing['2011q2'] = housing.iloc[:,139:142].mean(axis=1)
    housing['2011q3'] = housing.iloc[:,142:145].mean(axis=1)
    housing['2011q4'] = housing.iloc[:,145:148].mean(axis=1)
    housing['2012q1'] = housing.iloc[:,148:151].mean(axis=1)
    housing['2012q2'] = housing.iloc[:,151:154].mean(axis=1)
    housing['2012q3'] = housing.iloc[:,154:157].mean(axis=1)
    housing['2012q4'] = housing.iloc[:,157:160].mean(axis=1)
    housing['2013q1'] = housing.iloc[:,160:163].mean(axis=1)
    housing['2013q2'] = housing.iloc[:,163:166].mean(axis=1)
    housing['2013q3'] = housing.iloc[:,166:169].mean(axis=1)
    housing['2013q4'] = housing.iloc[:,169:172].mean(axis=1)
    housing['2014q1'] = housing.iloc[:,172:175].mean(axis=1)
    housing['2014q2'] = housing.iloc[:,175:178].mean(axis=1)
    housing['2014q3'] = housing.iloc[:,178:181].mean(axis=1)
    housing['2014q4'] = housing.iloc[:,181:184].mean(axis=1)
    housing['2015q1'] = housing.iloc[:,184:187].mean(axis=1)
    housing['2015q2'] = housing.iloc[:,187:190].mean(axis=1)
    housing['2015q3'] = housing.iloc[:,190:193].mean(axis=1)
    housing['2015q4'] = housing.iloc[:,193:196].mean(axis=1)
    housing['2016q1'] = housing.iloc[:,196:199].mean(axis=1)
    housing['2016q2'] = housing.iloc[:,199:202].mean(axis=1)
    housing['2016q3'] = housing.iloc[:,202:204].mean(axis=1)
    housing2 = housing.iloc[:,0:2]
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    housing2['State'].replace(states, inplace=True)
    housing3 = housing.iloc[:,204:]
    housing4 = housing2.join(housing3, how='outer')
    housing4 = housing4.set_index(['State','RegionName'])
    lst = list(housing4)
    qrt_rec_start = get_recession_start()
    number = lst.index(qrt_rec_start)
    number2 = number - 1
    qrt_bfr_rec_start = lst[number2]
    rec_bottom = get_recession_bottom()
    housing4['PriceRatio'] = housing4[qrt_bfr_rec_start].div(housing4[rec_bottom])
    ul = get_list_of_university_towns()
    housing4 = pd.merge(housing4.reset_index(), ul, on=ul.columns.tolist(), indicator='_flag', how='outer')
    group1=housing4[housing4['_flag']=='both']
    group2=housing4[housing4['_flag']!='both']
    university_town = group1['PriceRatio'].mean()
    non_university_town = group2['PriceRatio'].mean()
    tf = ttest_ind(group1['PriceRatio'], group2['PriceRatio'], nan_policy='omit').pvalue < .05
    p_value = ttest_ind(group1['PriceRatio'], group2['PriceRatio'], nan_policy='omit').pvalue
    (tf, ttest_ind(group1['PriceRatio'], group2['PriceRatio'], nan_policy='omit').pvalue, min(university_town, non_university_town))
    value = 'university town' if university_town < non_university_town else 'non-university town'
    return (tf, p_value, value)
